In [1]:
import random
import gym
from gym.spaces import Box
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import time
from CrimeWorld import CrimeWorld


EPISODES = 10

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.99    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
#         model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Conv2D(8, input_shape=(128,128,1), kernel_size=(8,8), strides=(4,4), padding='valid', activation='relu'))
        model.add(Conv2D(16, kernel_size=(4,4), strides=(2,2), padding='valid', activation='relu'))
#         model.add(Dense(24, activation='relu'))
#         model.add(Dense(self.action_size, activation='linear'))
        model.add(Flatten())
        model.add(Dense(64, activation='relu'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
#             print(next_state.shape)
            if not done:
                target = (reward + self.gamma *np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


In [2]:

if __name__ == "__main__":
    env = CrimeWorld()
    x = 60
    y = 60
    env.add_agent(x,y)
    state_size = env.get_state().shape[0]
    action_size = 5
    agent = DQNAgent(state_size, action_size)
    # agent.load("./save/cartpole-dqn.h5")
    done = False
    batch_size = 32


In [3]:

for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, state_size[0],state_size[1],1])
    for time in range(500):
        # env.render()
        action = agent.act(state)
        reward, next_state, done = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size[0],state_size[1],1])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
#         if done:
        print("episode: {}/{}, score: {}, e: {:.2}, {}" .format(e, EPISODES, time, agent.epsilon, done))
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

episode: 0/100, score: 0, e: 1.0, False
episode: 0/100, score: 1, e: 1.0, False
episode: 0/100, score: 2, e: 1.0, False
episode: 0/100, score: 3, e: 1.0, False
episode: 0/100, score: 4, e: 1.0, False
episode: 0/100, score: 5, e: 1.0, False
episode: 0/100, score: 6, e: 1.0, False
episode: 0/100, score: 7, e: 1.0, False
episode: 0/100, score: 8, e: 1.0, False
episode: 0/100, score: 9, e: 1.0, False
episode: 0/100, score: 10, e: 1.0, False
episode: 0/100, score: 11, e: 1.0, False
episode: 0/100, score: 12, e: 1.0, False
episode: 0/100, score: 13, e: 1.0, False
episode: 0/100, score: 14, e: 1.0, False
episode: 0/100, score: 15, e: 1.0, False
episode: 0/100, score: 16, e: 1.0, False
episode: 0/100, score: 17, e: 1.0, False
episode: 0/100, score: 18, e: 1.0, False
episode: 0/100, score: 19, e: 1.0, False
episode: 0/100, score: 20, e: 1.0, False
episode: 0/100, score: 21, e: 1.0, False
episode: 0/100, score: 22, e: 1.0, False
episode: 0/100, score: 23, e: 1.0, False
episode: 0/100, score: 24,

/home/smohnot/Desktop/project/CrimeWorld.py:189: RuntimeWarning: invalid value encountered in double_scalars
  p1 = A[i1,j] / totalA
/home/smohnot/Desktop/project/CrimeWorld.py:190: RuntimeWarning: invalid value encountered in double_scalars
  p2 = p1 + A[i2,j] / totalA
/home/smohnot/Desktop/project/CrimeWorld.py:191: RuntimeWarning: invalid value encountered in double_scalars
  p3 = p2 + A[i,j3] / totalA


episode: 0/100, score: 123, e: 0.63, False
episode: 0/100, score: 124, e: 0.63, False
episode: 0/100, score: 125, e: 0.63, False
episode: 0/100, score: 126, e: 0.62, False
episode: 0/100, score: 127, e: 0.62, False
episode: 0/100, score: 128, e: 0.62, False
episode: 0/100, score: 129, e: 0.61, False
episode: 0/100, score: 130, e: 0.61, False
episode: 0/100, score: 131, e: 0.61, False
episode: 0/100, score: 132, e: 0.61, False
episode: 0/100, score: 133, e: 0.6, False
episode: 0/100, score: 134, e: 0.6, False
episode: 0/100, score: 135, e: 0.6, False
episode: 0/100, score: 136, e: 0.59, False
episode: 0/100, score: 137, e: 0.59, False
episode: 0/100, score: 138, e: 0.59, False
episode: 0/100, score: 139, e: 0.58, False
episode: 0/100, score: 140, e: 0.58, False
episode: 0/100, score: 141, e: 0.58, False
episode: 0/100, score: 142, e: 0.58, False
episode: 0/100, score: 143, e: 0.57, False
episode: 0/100, score: 144, e: 0.57, False
episode: 0/100, score: 145, e: 0.57, False
episode: 0/100

episode: 0/100, score: 314, e: 0.24, False
episode: 0/100, score: 315, e: 0.24, False
episode: 0/100, score: 316, e: 0.24, False
episode: 0/100, score: 317, e: 0.24, False
episode: 0/100, score: 318, e: 0.24, False
episode: 0/100, score: 319, e: 0.24, False
episode: 0/100, score: 320, e: 0.24, False
episode: 0/100, score: 321, e: 0.23, False
episode: 0/100, score: 322, e: 0.23, False
episode: 0/100, score: 323, e: 0.23, False
episode: 0/100, score: 324, e: 0.23, False
episode: 0/100, score: 325, e: 0.23, False
episode: 0/100, score: 326, e: 0.23, False
episode: 0/100, score: 327, e: 0.23, False
episode: 0/100, score: 328, e: 0.23, False
episode: 0/100, score: 329, e: 0.23, False
episode: 0/100, score: 330, e: 0.22, False
episode: 0/100, score: 331, e: 0.22, False
episode: 0/100, score: 332, e: 0.22, False
episode: 0/100, score: 333, e: 0.22, False
episode: 0/100, score: 334, e: 0.22, False
episode: 0/100, score: 335, e: 0.22, False
episode: 0/100, score: 336, e: 0.22, False
episode: 0/

episode: 1/100, score: 5, e: 0.093, False
episode: 1/100, score: 6, e: 0.093, False
episode: 1/100, score: 7, e: 0.092, False
episode: 1/100, score: 8, e: 0.092, False
episode: 1/100, score: 9, e: 0.092, False
episode: 1/100, score: 10, e: 0.091, False
episode: 1/100, score: 11, e: 0.091, False
episode: 1/100, score: 12, e: 0.09, False
episode: 1/100, score: 13, e: 0.09, False
episode: 1/100, score: 14, e: 0.089, False
episode: 1/100, score: 15, e: 0.089, False
episode: 1/100, score: 16, e: 0.088, False
episode: 1/100, score: 17, e: 0.088, False
episode: 1/100, score: 18, e: 0.088, False
episode: 1/100, score: 19, e: 0.087, False
episode: 1/100, score: 20, e: 0.087, False
episode: 1/100, score: 21, e: 0.086, False
episode: 1/100, score: 22, e: 0.086, False
episode: 1/100, score: 23, e: 0.085, False
episode: 1/100, score: 24, e: 0.085, False
episode: 1/100, score: 25, e: 0.084, False
episode: 1/100, score: 26, e: 0.084, False
episode: 1/100, score: 27, e: 0.084, False
episode: 1/100, sc

...

episode: 17/100, score: 382, e: 0.01, False
episode: 17/100, score: 383, e: 0.01, False
episode: 17/100, score: 384, e: 0.01, False
episode: 17/100, score: 385, e: 0.01, False
episode: 17/100, score: 386, e: 0.01, False
episode: 17/100, score: 387, e: 0.01, False
episode: 17/100, score: 388, e: 0.01, False
episode: 17/100, score: 389, e: 0.01, False
episode: 17/100, score: 390, e: 0.01, False
episode: 17/100, score: 391, e: 0.01, False
episode: 17/100, score: 392, e: 0.01, False
episode: 17/100, score: 393, e: 0.01, False
episode: 17/100, score: 394, e: 0.01, False
episode: 17/100, score: 395, e: 0.01, False
episode: 17/100, score: 396, e: 0.01, False
episode: 17/100, score: 397, e: 0.01, False
episode: 17/100, score: 398, e: 0.01, False
episode: 17/100, score: 399, e: 0.01, False
episode: 17/100, score: 400, e: 0.01, False
episode: 17/100, score: 401, e: 0.01, False
episode: 17/100, score: 402, e: 0.01, False
episode: 17/100, score: 403, e: 0.01, False
episode: 17/100, score: 404, e: 

episode: 18/100, score: 72, e: 0.01, False
episode: 18/100, score: 73, e: 0.01, False
episode: 18/100, score: 74, e: 0.01, False
episode: 18/100, score: 75, e: 0.01, False
episode: 18/100, score: 76, e: 0.01, False
episode: 18/100, score: 77, e: 0.01, False
episode: 18/100, score: 78, e: 0.01, False
episode: 18/100, score: 79, e: 0.01, False
episode: 18/100, score: 80, e: 0.01, False
episode: 18/100, score: 81, e: 0.01, False
episode: 18/100, score: 82, e: 0.01, False
episode: 18/100, score: 83, e: 0.01, False
episode: 18/100, score: 84, e: 0.01, False
episode: 18/100, score: 85, e: 0.01, False
episode: 18/100, score: 86, e: 0.01, False
episode: 18/100, score: 87, e: 0.01, False
episode: 18/100, score: 88, e: 0.01, False
episode: 18/100, score: 89, e: 0.01, False
episode: 18/100, score: 90, e: 0.01, False
episode: 18/100, score: 91, e: 0.01, False
episode: 18/100, score: 92, e: 0.01, False
episode: 18/100, score: 93, e: 0.01, False
episode: 18/100, score: 94, e: 0.01, False
episode: 18

episode: 18/100, score: 260, e: 0.01, False
episode: 18/100, score: 261, e: 0.01, False
episode: 18/100, score: 262, e: 0.01, False
episode: 18/100, score: 263, e: 0.01, False
episode: 18/100, score: 264, e: 0.01, False
episode: 18/100, score: 265, e: 0.01, False
episode: 18/100, score: 266, e: 0.01, False
episode: 18/100, score: 267, e: 0.01, False
episode: 18/100, score: 268, e: 0.01, False
episode: 18/100, score: 269, e: 0.01, False
episode: 18/100, score: 270, e: 0.01, False
episode: 18/100, score: 271, e: 0.01, False
episode: 18/100, score: 272, e: 0.01, False
episode: 18/100, score: 273, e: 0.01, False
episode: 18/100, score: 274, e: 0.01, False
episode: 18/100, score: 275, e: 0.01, False
episode: 18/100, score: 276, e: 0.01, False
episode: 18/100, score: 277, e: 0.01, False
episode: 18/100, score: 278, e: 0.01, False
episode: 18/100, score: 279, e: 0.01, False
episode: 18/100, score: 280, e: 0.01, False
episode: 18/100, score: 281, e: 0.01, False
episode: 18/100, score: 282, e: 

episode: 18/100, score: 448, e: 0.01, False
episode: 18/100, score: 449, e: 0.01, False
episode: 18/100, score: 450, e: 0.01, False
episode: 18/100, score: 451, e: 0.01, False
episode: 18/100, score: 452, e: 0.01, False
episode: 18/100, score: 453, e: 0.01, False
episode: 18/100, score: 454, e: 0.01, False
episode: 18/100, score: 455, e: 0.01, False
episode: 18/100, score: 456, e: 0.01, False
episode: 18/100, score: 457, e: 0.01, False
episode: 18/100, score: 458, e: 0.01, False
episode: 18/100, score: 459, e: 0.01, False
episode: 18/100, score: 460, e: 0.01, False
episode: 18/100, score: 461, e: 0.01, False
episode: 18/100, score: 462, e: 0.01, False
episode: 18/100, score: 463, e: 0.01, False
episode: 18/100, score: 464, e: 0.01, False
episode: 18/100, score: 465, e: 0.01, False
episode: 18/100, score: 466, e: 0.01, False
episode: 18/100, score: 467, e: 0.01, False
episode: 18/100, score: 468, e: 0.01, False
episode: 18/100, score: 469, e: 0.01, False
episode: 18/100, score: 470, e: 

KeyboardInterrupt: 